# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#----------------------------------------------------------------------------------------
# DEPENDANCIES
#----------------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

#----------------------------------------------------------------------------------------
# IMPORT API KEY
#----------------------------------------------------------------------------------------
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#----------------------------------------------------------------------------------------
# READ CITIES CSV FILE
#----------------------------------------------------------------------------------------
weather_data_path = "../WeatherPy/cities_df.csv"
weather_data = pd.read_csv(weather_data_path)

#----------------------------------------------------------------------------------------
# DROP UNNAMED COLUMN
#----------------------------------------------------------------------------------------
weather_data = weather_data[['City', 'Country', 'Latitude', 'Longitude', 'Temperature',
       'Humidity', 'Cloudiness', 'Wind_Speed']]

#----------------------------------------------------------------------------------------
# PREVIEW DATAFRAME
#----------------------------------------------------------------------------------------
weather_data.head()

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed
0,Dem'yanovo,RU,60.3511,47.0842,2.43,81,98,1.90
1,Hardoī,IN,27.4167,80.1167,39.40,10,0,5.44
2,Molina de Segura,ES,38.0546,-1.2076,13.22,80,73,1.34
3,Aiton,RO,46.6833,23.7333,10.15,82,0,3.09
4,Passy,FR,45.9203,6.6877,8.04,99,100,1.71


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#----------------------------------------------------------------------------------------
# STORE REQUIRED LAT/LONG AND HUMIDITY DATA IN VARIABLES
#----------------------------------------------------------------------------------------
locations = weather_data[["Latitude", "Longitude"]]
humidity = weather_data["Humidity"]

In [4]:
#----------------------------------------------------------------------------------------
# CREATE HEATMAP
#----------------------------------------------------------------------------------------
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
                                 
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#----------------------------------------------------------------------------------------------------
# FIND CITIES WITH PERFECT CONDITIONS. DROP ANY ROWS THAT DON"T CONTAIN ALL THREE CONDITIONS BELOW
#--- TEMPERATURE BETWEEN 21 & 27 DEGREES CENTIGRADE
#--- WIND SPEED LESS THAN 10 MPH
#--- ZERO CLOUDINESS
#----------------------------------------------------------------------------------------------------

# Filter the dataframe
perfect_weather_df = weather_data[(weather_data['Temperature'] >= 21) & (weather_data['Temperature'] < 23) & (weather_data['Wind_Speed'] < 10) & (weather_data['Cloudiness'] == 0)]

# Visualize
print(len(perfect_weather_df))
perfect_weather_df.head()


10


,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed
77,Goldenrod,US,28.6103,-81.2887,22.76,84,0,3.09
156,Samandağ,TR,36.0842,35.9771,21.29,49,0,1.10
199,Bellview,US,30.4616,-87.3150,21.56,97,0,2.68
239,Atotonilquillo,MX,20.4000,-103.1333,22.07,40,0,2.71
266,Guadalajara,MX,20.6667,-103.3333,21.87,33,0,3.60


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#----------------------------------------------------------------------------------------
# STORE CITIES WITH DESIRED WEATHER INTO VARIABLE NAMED 'HOTEL_DF' AND ADD NEW COLUMN
#----------------------------------------------------------------------------------------
hotel_df = perfect_weather_df
hotel_df["Hotel Name"] = ''
hotel_df.head(20)


C:\Users\rjtur\AppData\Local\Temp/ipykernel_280/3716367173.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ''


,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed,Hotel Name
77,Goldenrod,US,28.6103,-81.2887,22.76,84,0,3.09,
156,Samandağ,TR,36.0842,35.9771,21.29,49,0,1.10,
199,Bellview,US,30.4616,-87.3150,21.56,97,0,2.68,
239,Atotonilquillo,MX,20.4000,-103.1333,22.07,40,0,2.71,
266,Guadalajara,MX,20.6667,-103.3333,21.87,33,0,3.60,
302,Gonzalez,US,30.5816,-87.2914,21.97,97,0,1.54,
334,Santa Ana,US,33.7456,-117.8678,22.29,32,0,0.45,
401,Jacksonville,US,30.3322,-81.6556,22.00,90,0,1.54,
460,Puente Grande,MX,20.5667,-103.1500,22.50,33,0,3.60,
513,Fylla,GR,38.4333,23.6833,22.42,75,0,2.06,


In [7]:
#----------------------------------------------------------------------------------------
# SET PARAMETERS TO SEARCH FOR HOTELS WITHIN 5000 METERS
# HIT THE GOOGLE PLACES API FOR EACH CITY"S COORDINATES
# STORE THE FIRST HOTEL RESULT INTO THE DATAFRAME
#----------------------------------------------------------------------------------------

# Empty list to store hotel names
hotel_list = []

# Set list of city names for loop
cities_data = hotel_df["City"]

# Loop through each city in the list of desired cities
for city in cities_data:

    # Set latitude and longitude values
    lat = hotel_df[hotel_df["City"] == city]["Latitude"].values[0]
    long = hotel_df[hotel_df["City"] == city]["Longitude"].values[0]

    # Set parameters
    target_coordinates = str(lat) + ", " + str(long)
    target_radius = 5000
    target_search = 'hotel'
    target_type = 'lodging'

    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    # convert response to json
    hotels_data = response.json()   

    # Append hotel name to hotel_list
    hotel_list.append(hotels_data["results"][0]["name"])

# Add hotel names to hotel_df    
hotel_df["Hotel Name"] = hotel_list

# Display df
hotel_df

C:\Users\rjtur\AppData\Local\Temp/ipykernel_280/3868740856.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = hotel_list


,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed,Hotel Name
77,Goldenrod,US,28.6103,-81.2887,22.76,84,0,3.09,Colonial Village
156,Samandağ,TR,36.0842,35.9771,21.29,49,0,1.10,Anadolu Palace Hotel
199,Bellview,US,30.4616,-87.3150,21.56,97,0,2.68,Hospitality Inn Management Co
239,Atotonilquillo,MX,20.4000,-103.1333,22.07,40,0,2.71,Antigua Estación de Tren
266,Guadalajara,MX,20.6667,-103.3333,21.87,33,0,3.60,Hotel Aranzazú
302,Gonzalez,US,30.5816,-87.2914,21.97,97,0,1.54,Rosan Plaza Hotel
334,Santa Ana,US,33.7456,-117.8678,22.29,32,0,0.45,"Motel 6 Santa Ana, CA"
401,Jacksonville,US,30.3322,-81.6556,22.00,90,0,1.54,Hyatt Regency Jacksonville Riverfront
460,Puente Grande,MX,20.5667,-103.1500,22.50,33,0,3.60,Casa con alberca y terraza cabaña san Antonio
513,Fylla,GR,38.4333,23.6833,22.42,75,0,2.06,Lefkadi


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))